# Project Title
### Data Engineering Capstone Project

#### Project Summary
The purpose of the capstone project for data engineering is to enable students to integrate the knowledge and skills they have gained throughout the program. It is a significant component of a learner's portfolio that will contribute to achieving their career objectives in the field of data engineering. The primary objective of the project was to create an ETL pipeline for combining I94 immigration, global land temperatures, and US demographics datasets into an analytics database to analyze immigration events. The resulting analytics database can be used to identify immigration patterns to the US, such as whether people from countries with warmer or colder climates tend to immigrate to the US in greater numbers.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import configparser
import datetime as dt

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import *

import plotly.plotly as py
import plotly.graph_objs as go
import requests
requests.packages.urllib3.disable_warnings()

import utility
import etl_functions

import importlib
importlib.reload(utility)
from utility import visualize_missing_values, clean_immigration, clean_temperature_data
from utility import clean_demographics_data, print_formatted_float

### Load Data Configuration

In [ ]:
config = configparser.ConfigParser()
config.read('config.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

### Create a Spark Session

In [ ]:
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
The analytics database creation process involves several steps, including:

- Using Spark to load the datasets into dataframes.
- Conducting exploratory data analysis on the I94 immigration, demographics, and global land temperatures datasets to identify missing values and develop strategies for data cleaning.
- Performing data cleaning functions on all datasets.

Creating dimension tables, including:
- The immigration calendar dimension table, which is created from the I94 immigration dataset and linked to the fact table through the arrdate field.
- The country dimension table, which is created from the I94 immigration and global temperatures datasets and linked to the fact table through the country of residence code, allowing analysts to explore correlations between climate and immigration to US states.
- The USA demographics dimension table, which is created from the US cities demographics data and linked to the fact table through the state code field.
- Creating the fact table from the cleaned I94 immigration dataset and the visa_type dimension.


Amazon S3 and Apache Spark are the technologies used in this project. The data will be read and staged from the customer's repository using Spark. Although the project is implemented on this notebook, provisions have been made to run the ETL on a spark cluster through the etl.py script. This script reads data from S3 and creates fact and dimension tables through Spark, which are then loaded back into S3.





#### Describe and Gather Data 
##### Immigration Data
For many years, foreign visitors (such as business visitors, tourists, and foreign students) who entered the United States lawfully were issued the I-94 Form (Arrival/Departure Record) by U.S. immigration officers. The I-94 was a small white paper form that was given to travelers on arrival flights by cabin crews or by U.S. Customs and Border Protection at the time of entry into the United States. It contained important information such as the traveler's immigration category, port of entry, date of entry into the United States, status expiration date, and a unique 11-digit identifying number. Its primary purpose was to document the traveler's lawful admission to the United States.

The I-94 dataset is the primary dataset used in this project, and it is available in the SAS binary database storage format sas7bdat. The directory *../../data/18-83510-I94-Data-2016/* contains a file for each month of 2016, totaling 12 datasets with over 40 million rows (40,790,529) and 28 columns.

In [ ]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = spark.read.format('com.github.saurfang.sas.spark').load(fname)

In [ ]:
df_immigration.limit(5).toPandas()

<b><i>Data dictionary</i></b>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">cicid</td><td class="tg-0pky">Unique record ID</td>
 <tr><td class="tg-0pky">i94yr</td><td class="tg-0pky">4 digit year</td>
 <tr><td class="tg-0pky">i94mon</td><td class="tg-0pky">Numeric month</td>
 <tr><td class="tg-0pky">i94cit</td><td class="tg-0pky">3 digit code for immigrant country of birth</td>
 <tr><td class="tg-0pky">i94res</td><td class="tg-0pky">3 digit code for immigrant country of residence </td>
 <tr><td class="tg-0pky">i94port</td><td class="tg-0pky">Port of admission</td>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival Date in the USA</td>
 <tr><td class="tg-0pky">i94mode</td><td class="tg-0pky">Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)</td>
 <tr><td class="tg-0pky">i94addr</td><td class="tg-0pky">USA State of arrival</td>
 <tr><td class="tg-0pky">depdate</td><td class="tg-0pky">Departure Date from the USA</td>
 <tr><td class="tg-0pky">i94bir</td><td class="tg-0pky">Age of Respondent in Years</td>
 <tr><td class="tg-0pky">i94visa</td><td class="tg-0pky">Visa codes collapsed into three categories</td>
 <tr><td class="tg-0pky">count</td><td class="tg-0pky">Field used for summary statistics</td>
 <tr><td class="tg-0pky">dtadfile</td><td class="tg-0pky">Character Date Field - Date added to I-94 Files</td>
 <tr><td class="tg-0pky">visapost</td><td class="tg-0pky">Department of State where where Visa was issued </td>
 <tr><td class="tg-0pky">occup</td><td class="tg-0pky">Occupation that will be performed in U.S</td>
 <tr><td class="tg-0pky">entdepa</td><td class="tg-0pky">Arrival Flag - admitted or paroled into the U.S.</td>
 <tr><td class="tg-0pky">entdepd</td><td class="tg-0pky">Departure Flag - Departed, lost I-94 or is deceased</td>
 <tr><td class="tg-0pky">entdepu</td><td class="tg-0pky">Update Flag - Either apprehended, overstayed, adjusted to perm residence</td>
 <tr><td class="tg-0pky">matflag</td><td class="tg-0pky">Match flag - Match of arrival and departure records</td>
 <tr><td class="tg-0pky">biryear</td><td class="tg-0pky">4 digit year of birth</td>
 <tr><td class="tg-0pky">dtaddto</td><td class="tg-0pky">Character Date Field - Date to which admitted to U.S. (allowed to stay until)</td>
 <tr><td class="tg-0pky">gender</td><td class="tg-0pky">Non-immigrant sex</td>
 <tr><td class="tg-0pky">insnum</td><td class="tg-0pky">INS number</td>
 <tr><td class="tg-0pky">airline</td><td class="tg-0pky">Airline used to arrive in U.S.</td>
 <tr><td class="tg-0pky">admnum</td><td class="tg-0pky">Admission Number</td>
 <tr><td class="tg-0pky">fltno</td><td class="tg-0pky">Flight number of Airline used to arrive in U.S.</td>
 <tr><td class="tg-0pky">visatype</td><td class="tg-0pky">Class of admission legally admitting the non-immigrant to temporarily stay in U.S.</td>
</table>


##### Global Land Temperature Data

There are various organizations that gather data on climate trends. The three most commonly referenced datasets for land and ocean temperatures are NOAA's MLOST, NASA's GISTEMP, and the UK's HadCrut.

Berkeley Earth, which is affiliated with Lawrence Berkeley National Laboratory, has compiled and repackaged data from a newer compilation. The Berkeley Earth Surface Temperature Study is a collection of 1.6 billion temperature reports from 16 existing archives. It is well-organized and allows for filtering of interesting subsets, such as by country. They provide both the source data and the code for the transformations applied. Additionally, they use methods that enable weather observations from shorter time series to be included, resulting in fewer observations being discarded.

Although several files are available in the original dataset from Kaggle, this capstone project will only utilize the GlobalLandTemperaturesByCity dataset.





In [ ]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = spark.read.csv(file_name, header=True, inferSchema=True)

In [ ]:
df_temperature.limit(5).toPandas()

<b><i>Data dictionary</i></b>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">dt</td><td class="tg-0pky">Date</td>
 <tr><td class="tg-0pky">AverageTemperature</td><td class="tg-0pky">Global average land temperature in celsius</td>
 <tr><td class="tg-0pky">AverageTemperatureUncertainty</td><td class="tg-0pky">95% confidence interval around the average</td>
 <tr><td class="tg-0pky">City</td><td class="tg-0pky">Name of City</td>
 <tr><td class="tg-0pky">Country</td><td class="tg-0pky">Name of Country</td>
 <tr><td class="tg-0pky">Latitude</td><td class="tg-0pky">City Latitude</td>
 <tr><td class="tg-0pky">Longitude</td><td class="tg-0pky">City Longitude</td>
</table>

##### Airport Data

The airport codes can refer to either the IATA airport code, a three-letter code used in passenger reservation, ticketing, and baggage-handling systems, or the ICAO airport code, a four-letter code used by ATC systems and for airports that do not have an IATA airport code (as per Wikipedia).

The list of airport codes used around the world is available in the airport-codes.csv file. The data was downloaded from a public domain source http://ourairports.com/data/, which compiled the data from multiple different sources. The attributes in the dataset are identified in the datapackage description, and some of the columns contain attributes that identify airport locations and other codes (IATA, local if exist) that are relevant to the identification of an airport. The original source URL for this data is http://ourairports.com/data/airports.csv and is stored in the archive/data.csv file.

In [ ]:
file_name = "airport-codes_csv.csv"
df_airport = spark.read.csv(file_name,  inferSchema=True, header=True, sep=';')

In [ ]:
df_airport.limit(5).toPandas()

<b><i>Data dictionary</i></b>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">ident</td><td class="tg-0pky">Unique identifier</td>
 <tr><td class="tg-0pky">type</td><td class="tg-0pky">Type of the airport</td>
 <tr><td class="tg-0pky">name</td><td class="tg-0pky">Airport Name</td>
 <tr><td class="tg-0pky">elevation_ft</td><td class="tg-0pky">Altitude of the airport</td>
 <tr><td class="tg-0pky">continent</td><td class="tg-0pky">Continent</td>
 <tr><td class="tg-0pky">iso_country</td><td class="tg-0pky">ISO code of the country of the airport</td>
 <tr><td class="tg-0pky">iso_region</td><td class="tg-0pky">ISO code for the region of the airport</td>
 <tr><td class="tg-0pky">municipality</td><td class="tg-0pky">City where the airport is located</td>
 <tr><td class="tg-0pky">gps_code</td><td class="tg-0pky">GPS code of the airport</td>
 <tr><td class="tg-0pky">iata_code</td><td class="tg-0pky">IATA code of the airport</td>
 <tr><td class="tg-0pky">local_code</td><td class="tg-0pky">Local code of the airport</td>
 <tr><td class="tg-0pky">coordinates</td><td class="tg-0pky">GPS coordinates of the airport</td>
</table>

The airport dataset was not found to be a useful source for analysis in our model because we were unable to join it with the main table immigration. We could not find a valid and consistent key that could be used to link the two tables together. None of the codes in the airport dataset, such as ident, gps_code, iata_code or local_code, appeared to match the columns in the immigration fact table. Therefore, we decided not to use the airport dataset in our model.

##### US City Demographic Data

The dataset, obtained from OpenSoft, provides demographic information on all US cities and census-designated places with a population of 65,000 or more. The source of the data is the US Census Bureau's 2015 American Community Survey.

In [ ]:
file_name = "us-cities-demographics.csv"
df_us_cities_demographics = spark.read.csv(file_name, inferSchema=True, header=True, sep=';')

In [ ]:
df_us_cities_demographics.limit(5).toPandas()

<b><i>Data dictionary</i></b>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">City</td><td class="tg-0pky">City Name</td>
 <tr><td class="tg-0pky">State</td><td class="tg-0pky">US State where city is located</td>
 <tr><td class="tg-0pky">Median Age</td><td class="tg-0pky">Median age of the population</td>
 <tr><td class="tg-0pky">Male Population</td><td class="tg-0pky">Count of male population</td>
 <tr><td class="tg-0pky">Female Population</td><td class="tg-0pky">Count of female population</td>
 <tr><td class="tg-0pky">Total Population</td><td class="tg-0pky">Count of total population</td>
 <tr><td class="tg-0pky">Number of Veterans</td><td class="tg-0pky">Count of total Veterans</td>
 <tr><td class="tg-0pky">Foreign born</td><td class="tg-0pky">Count of residents of the city that were not born in the city</td>
 <tr><td class="tg-0pky">Average Household Size</td><td class="tg-0pky">Average city household size</td>
 <tr><td class="tg-0pky">State Code</td><td class="tg-0pky">Code of the US state</td>
 <tr><td class="tg-0pky">Race</td><td class="tg-0pky">Respondent race</td>
 <tr><td class="tg-0pky">Count</td><td class="tg-0pky">Count of city's individual per race</td>
</table>

### Step 2: Explore and Assess the Data
#### Explore the Data 

##### Missing values

In [ ]:
# immigration Data

utility.visualize_missing_values_spark(df_immigration)

In [ ]:
# Global temperature Data
temperature_df = temperature_df.withColumn("dt",col("dt").cast(StringType())) # convert dt column type to string
utility.visualize_missing_values_spark(temperature_df2)

In [ ]:
# US City Demographic Data
utility.visualize_missing_values_spark(df_us_cities_demographics)

*The height of each bar represents the percentage of missing values in that column. If a column has no missing values, the corresponding bar will be completely unfilled.*


#### Cleaning Steps

##### Drop columns with more than 90% of missing values

In [ ]:
# columns with over 90% missing values
cols = ['occup', 'entdepu','insnum']

# drop these columns
df_immigration = df_immigration.drop(*cols)

##### Drop duplicates

In [ ]:
df_immigration = df_immigration.dropDuplicates()

In [ ]:
df_temperature = df_temperature.dropDuplicates()

In [ ]:
df_us_cities_demographics = df_us_cities_demographics.dropDuplicates()

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
![Database schema](conceptual_model.png)

The country dimension table is comprised of data from two datasets: the global land temperatures by city and the immigration datasets. This combination of data enables analysts to investigate the correlations between global land temperatures and immigration patterns to the US.

The US demographics dimension table is sourced from the demographics dataset and is linked to the immigration fact table at the US state level. This dimension allows analysts to gain insights into migration patterns based on demographics and overall population of states. For example, one could explore if more visitors come to populous states on a monthly basis. By leveraging this data model, a dashboard can be created that offers granular information on visits to the US, potentially driving data-driven decision making within the tourism and immigration departments at the state level.

The visa type dimension table is sourced from the immigration datasets and is linked to the immigration fact table via the visa_type_key.

The immigration fact table is the centerpiece of the data model. It's comprised of data from the immigration datasets and contains keys that link to the dimension tables. Detailed information on the data that makes up the fact table can be found in the data dictionary of the immigration dataset.




#### 3.2 Mapping Out Data Pipelines
The pipeline steps are as follows:

* Load the datasets
* Clean the I94 Immigration data to create Spark dataframe for each month
* Create visa_type dimension table
* Create calendar dimension table
* Extract clean global temperatures data
* Create country dimension table
* Create immigration fact table
* Load demographics data
* Clean demographics data
* Create demographic dimension table

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

##### Create the immigration calendar dimension table
---

In [ ]:
def create_immigration_calendar_dimension(df, output_data):
    """This function creates an immigration calendar based on arrival date
    
    :param df: spark dataframe of immigration events
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # create a udf to convert arrival date in SAS format to datetime object
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    # create initial calendar df from arrdate column
    calendar_df = df.select(['arrdate']).withColumn("arrdate", get_datetime(df.arrdate)).distinct()
    
    # expand df by adding other calendar columns
    calendar_df = calendar_df.withColumn('arrival_day', dayofmonth('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_week', weekofyear('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_month', month('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_year', year('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_weekday', dayofweek('arrdate'))

    # create an id field in calendar df
    calendar_df = calendar_df.withColumn('id', monotonically_increasing_id())
    
    # write the calendar dimension to parquet file
    partition_columns = ['arrival_year', 'arrival_month', 'arrival_week']
    calendar_df.write.parquet(output_data + "immigration_calendar", partitionBy=partition_columns, mode="overwrite")
    
    return calendar_df

In [ ]:
output_data = "tables/"

In [ ]:
calendar_df = create_immigration_calendar_dimension(new_immigration_df, output_data)

##### Create the country dimension table
---

In [ ]:
def create_country_dimension_table(df, temp_df, output_data):
    """This function creates a country dimension from the immigration and global land temperatures data.
    
    :param df: spark dataframe of immigration events
    :temp_df: spark dataframe of global land temperatures data.
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # get the aggregated temperature data
    agg_temp = utility.aggregate_temperature_data(temp_df).toPandas()
    # load the i94res to country mapping data
    mapping_codes = pd.read_csv('i94res.csv')
    
    @udf('string')
    def get_country_average_temperature(name):
        print("Processing: ", name)
        avg_temp = agg_temp[agg_temp['Country']==name]['average_temperature']
        
        if not avg_temp.empty:
            return str(avg_temp.iloc[0])
        
        return None
    
    @udf()
    def get_country_name(code):
        name = mapping_codes[mapping_codes['code']==code]['Name'].iloc[0]
        
        if name:
            return name.title()
        return None
        
    # select and rename i94res column
    dim_df = df.select(['i94res']).distinct() \
                .withColumnRenamed('i94res', 'country_code')
    
    # create country_name column
    dim_df = dim_df.withColumn('country_name', get_country_name(dim_df.country_code))
    
    # create average_temperature column
    dim_df = dim_df.withColumn('average_temperature', get_country_average_temperature(dim_df.country_name))
    
    # write the dimension to a parquet file
    dim_df.write.parquet(output_data + "country", mode="overwrite")
    
    return dim_df

In [ ]:
country_dim_f = create_country_dimension_table(new_immigration_df, new_temperature_df, output_data)

In [ ]:
country_dim_f.show(5)

##### Create the visa type dimension table
---

In [ ]:
def create_visa_type_dimension_table(df, output_data):
    """This function creates a visa type dimension from the immigration data.
    
    :param df: spark dataframe of immigration events
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # create visatype df from visatype column
    visatype_df = df.select(['visatype']).distinct()
    
    # add an id column
    visatype_df = visatype_df.withColumn('visa_type_key', monotonically_increasing_id())
    
    # write dimension to parquet file
    visatype_df.write.parquet(output_data + "visatype", mode="overwrite")
    
    return visatype_df

def get_visa_type_dimension(output_data):
    return spark.read.parquet(output_data + "visatype")

In [ ]:
# test create visa_type dimension function
visatype_df = create_visa_type_dimension_table(new_immigration_df, output_data)
visatype_df.show(n=5)

##### Create the demographics dimension table
---

In [ ]:
def create_demographics_dimension_table(df, output_data):
    """This function creates a us demographics dimension table from the us cities demographics data.
    
    :param df: spark dataframe of us demographics survey data
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing demographics dimension
    """
    dim_df = df.withColumnRenamed('Median Age','median_age') \
            .withColumnRenamed('Male Population', 'male_population') \
            .withColumnRenamed('Female Population', 'female_population') \
            .withColumnRenamed('Total Population', 'total_population') \
            .withColumnRenamed('Number of Veterans', 'number_of_veterans') \
            .withColumnRenamed('Foreign-born', 'foreign_born') \
            .withColumnRenamed('Average Household Size', 'average_household_size') \
            .withColumnRenamed('State Code', 'state_code')
    # lets add an id column
    dim_df = dim_df.withColumn('id', monotonically_increasing_id())
    
    # write dimension to parquet file
    dim_df.write.parquet(output_data + "demographics", mode="overwrite")
    
    return dim_df

In [ ]:
demographics_dim_df = create_demographics_dimension_table(new_demographics_df, output_data)
demographics_dim_df.limit(5).toPandas()

##### Create the immigration fact table
---

In [ ]:
def create_immigration_fact_table(df, output_data):
    """This function creates an country dimension from the immigration and global land temperatures data.
    
    :param df: spark dataframe of immigration events
    :param visa_type_df: spark dataframe of global land temperatures data.
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # get visa_type dimension
    dim_df = get_visa_type_dimension(output_data).toPandas()
    
    @udf('string')
    def get_visa_key(visa_type):
        """user defined function to get visa key
        
        :param visa_type: US non-immigrant visa type
        :return: corresponding visa key
        """
        key_series = dim_df[dim_df['visatype']==visa_type]['visa_type_key']
        
        if not key_series.empty:
            return str(key_series.iloc[0])
        
        return None
    
    # create a udf to convert arrival date in SAS format to datetime object
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    # rename columns to align with data model
    df = df.withColumnRenamed('cicid','record_id') \
            .withColumnRenamed('i94res', 'country_residence_code') \
            .withColumnRenamed('i94addr', 'state_code') 
    
    # create visa_type key
    df = df.withColumn('visa_type_key', get_visa_key('visatype'))
    
    # convert arrival date into datetime object
    df = df.withColumn("arrdate", get_datetime(df.arrdate))
    
    # write dimension to parquet file
    df.write.parquet(output_data + "immigration_fact", mode="overwrite")
    
    return df

In [ ]:
immigration_fact_df = create_immigration_fact_table(new_immigration_df, output_data)

In [ ]:
def run_pipeline():
    # load data
    
    # run cleaning functions
    
    # create fact and dimension tables
    
    

#### 4.2 Data Quality Checks
The data quality checks ensures that the ETL has created fact and dimension tables with adequate records. 

#### 4.2 Data Quality Checks
The data quality checks ensures that the ETL has created fact and dimension tables with adequate records. 

In [ ]:
table_dfs = {
    'immigration_fact': immigration_fact_df,
    'visa_type_dim': visatype_df,
    'calendar_dim': calendar_df,
    'usa_demographics_dim': demographics_dim_df,
    'country_dim': country_dim_f
}
for table_name, table_df in table_dfs.items():
    # quality check for table
    etl_functions.quality_checks(table_df, table_name)

#### 4.3 Data dictionary 



##### Fact Table - data dictionary

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">record_id</td><td class="tg-0pky">Unique record ID</td></tr>
 <tr><td class="tg-0pky">country_residence_code</td><td class="tg-0pky">3 digit code for immigrant country of residence </td></tr>    
 <tr><td class="tg-0pky">visa_type_key</td><td class="tg-0pky">A numerical key that links to the visa_type dimension table</td></tr>
 <tr><td class="tg-0pky">state_code</td><td class="tg-0pky">US state of arrival</td></tr>
 <tr><td class="tg-0pky">i94yr</td><td class="tg-0pky">4 digit year</td></tr>
 <tr><td class="tg-0pky">i94mon</td><td class="tg-0pky">Numeric month</td></tr>
 <tr><td class="tg-0pky">i94port</td><td class="tg-0pky">Port of admission</td></tr>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival Date in the USA</td></tr>
 <tr><td class="tg-0pky">i94mode</td><td class="tg-0pky">Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)</td></tr>
 <tr><td class="tg-0pky">i94addr</td><td class="tg-0pky">USA State of arrival</td></tr>
 <tr><td class="tg-0pky">depdate</td><td class="tg-0pky">Departure Date from the USA</td></tr>
 <tr><td class="tg-0pky">i94bir</td><td class="tg-0pky">Age of Respondent in Years</td></tr>
 <tr><td class="tg-0pky">i94visa</td><td class="tg-0pky">Visa codes collapsed into three categories</td></tr>
 <tr><td class="tg-0pky">count</td><td class="tg-0pky">Field used for summary statistics</td></tr>
 <tr><td class="tg-0pky">dtadfile</td><td class="tg-0pky">Character Date Field - Date added to I-94 Files</td></tr>
 <tr><td class="tg-0pky">visapost</td><td class="tg-0pky">Department of State where where Visa was issued </td></tr>
 <tr><td class="tg-0pky">occup</td><td class="tg-0pky">Occupation that will be performed in U.S</td></tr>
 <tr><td class="tg-0pky">entdepa</td><td class="tg-0pky">Arrival Flag - admitted or paroled into the U.S.</td></tr>
 <tr><td class="tg-0pky">entdepd</td><td class="tg-0pky">Departure Flag - Departed, lost I-94 or is deceased</td></tr>
 <tr><td class="tg-0pky">entdepu</td><td class="tg-0pky">Update Flag - Either apprehended, overstayed, adjusted to perm residence</td></tr>
 <tr><td class="tg-0pky">matflag</td><td class="tg-0pky">Match flag - Match of arrival and departure records</td></tr>
 <tr><td class="tg-0pky">biryear</td><td class="tg-0pky">4 digit year of birth</td></tr>
 <tr><td class="tg-0pky">dtaddto</td><td class="tg-0pky">Character Date Field - Date to which admitted to U.S. (allowed to stay until)</td></tr>
 <tr><td class="tg-0pky">gender</td><td class="tg-0pky">Non-immigrant sex</td></tr>
</table>

##### Country Dimension Table - data dictionary
<p>  
<i>The country code and country_name fields come from the labels description SAS file while the average_temperature data comes from the global land temperature by cities data.</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">country_code</td><td class="tg-0pky">Unique country code</td></tr>
 <tr><td class="tg-0pky">country_name</td><td class="tg-0pky">Name of country</td></tr>    
 <tr><td class="tg-0pky">average_temperature</td><td class="tg-0pky">Average temperature of country</td></tr>
</table>

##### Visa Type Dimension Table - data dictionary
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">visa_type_key</td><td class="tg-0pky">Unique id for each visa issued</td></tr>
 <tr><td class="tg-0pky">visa_type</td><td class="tg-0pky">Name of visa</td></tr>
</table>

 ##### Immigration Calendar Dimension Table - data dictionary
<p>
<i>The whole of this dataset comes from the immigration dataset.</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">id</td><td class="tg-0pky">Unique id</td></tr>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival date into US</td></tr>    
 <tr><td class="tg-0pky">arrival_year</td><td class="tg-0pky">Arrival year into US</td></tr>
 <tr><td class="tg-0pky">arrival_month</td><td class="tg-0pky">Arrival MonthS</td></tr>
 <tr><td class="tg-0pky">arrival_day</td><td class="tg-0pky">Arrival Day</td></tr>
 <tr><td class="tg-0pky">arrival_week</td><td class="tg-0pky">Arrival Week</td></tr>
 <tr><td class="tg-0pky">arrival_weekday</td><td class="tg-0pky">Arrival WeekDay</td></tr>
</table>

##### US Demographics Dimension Table - data dictionary
<p>
<i>The whole of this dataset comes from the us cities demographics data.</i>
</p>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">id</td><td class="tg-0pky">Record id</td>
 <tr><td class="tg-0pky">state_code</td><td class="tg-0pky">US state code </td>
 <tr><td class="tg-0pky">City</td><td class="tg-0pky">City Name</td>
 <tr><td class="tg-0pky">State</td><td class="tg-0pky">US State where city is located</td>
 <tr><td class="tg-0pky">Median Age</td><td class="tg-0pky">Median age of the population</td>
 <tr><td class="tg-0pky">Male Population</td><td class="tg-0pky">Count of male population</td>
 <tr><td class="tg-0pky">Female Population</td><td class="tg-0pky">Count of female population</td>
 <tr><td class="tg-0pky">Total Population</td><td class="tg-0pky">Count of total population</td>
 <tr><td class="tg-0pky">Number of Veterans</td><td class="tg-0pky">Count of total Veterans</td>
 <tr><td class="tg-0pky">Foreign born</td><td class="tg-0pky">Count of residents of the city that were not born in the city</td>
 <tr><td class="tg-0pky">Average Household Size</td><td class="tg-0pky">Average city household size</td>
 <tr><td class="tg-0pky">Race</td><td class="tg-0pky">Respondent race</td>
 <tr><td class="tg-0pky">Count</td><td class="tg-0pky">Count of city's individual per race</td>
</table>

#### Step 5: Complete Project Write Up
The reason for selecting tools and technologies for the project:
- Apache Spark was chosen because of its capability to handle large amounts of data in multiple file formats.
- Apache Spark provides a high-speed, comprehensive analytics engine for big data.
- Spark's APIs are user-friendly and allow for the processing of large datasets.

Suggestion for the frequency of data updates and explanation:
- The current I94 immigration data is refreshed on a monthly basis, and therefore the data will be updated monthly.

Explanation of how the approach would vary under the given circumstances:
- If the data increased by 100x, we would consider increasing the number of nodes in our cluster while Spark could manage the increase.
- If the data populated a dashboard that required daily updates by 7am, we would schedule and execute data pipelines using Apache Airflow.
- If the database needed to be accessed by 100+ people, we would relocate our analytics database to Amazon Redshift.